<img src='letscodebr_cover.jpeg' align='left' width=100%/>

# Aulas 4: Pandas - Agregação.

## Intro  - Split, Apply, Combine

Separar um conjunto de dados em categorias e aplicar uma função a cada grupo, que pode ser agregação, transformação ou filtro, é uma etapa muito comum em um fluxo de trabalho de análise de dados.

Depois de carregar e preparar um conjunto de dados, podemos precisar calcular estatísticas de grupo ou, possivelmente, tabelas dinâmicas para gerar relatórios ou visualizações.

`pandas` fornece métodos que nos permitem realizar essas tarefas naturalmente.

Nestes guias, aprenderemos a

* Dividir um objeto `pandas` em partes usando uma ou mais keys

* Calcular medidas de resumo em grupos, como quantidade, média, desvio padrão ou qualquer função definida pelo usuário

* Aplicar transformações por grupos.

* Construir tabelas dinâmicas

## Dataset

O [GCBA](https://www.buenosaires.gob.ar/) realiza pesquisas com turistas que vêm aos centros de atendimento. Pergunta-se o motivo da consulta, os dias da viagem, o país de origem, entre outros.

O conjunto de dados está publicamente acessível no portal de dados abertos do [GCBA](https://dados.buenosaires.gob.ar/dadosset/encuesta-centros-atencion-turistica-cat).

Neste guia usaremos o dataset Resultado de pesquisas em Centros de Atenção ao Turista (CAT) em 2017-2018.

## Problema

A partir dos dados das consultas aos postos de turismo da cidade de Buenos Aires, vamos responder a perguntas sobre o país de origem dos turistas, algumas medidas estatísticas sobre o número de dias de permanência na cidade e o número de visitantes.

## GroupBy

Podemos descrever as operações em grupos com o termo *split-apply-combine*.

Na primeira etapa do processo, os dados em um objeto `pandas` (uma instância de `Series` ou de `DataFrame`) se dividem em grupos (*split*) com base em uma ou mais keys que definimos. Esta divisão é feita por linhas (axis = 0) ou por colunas (axis = 1).

Como um segundo estágio, aplicamos uma função a cada um dos grupos (*apply*) resultando em um novo valor por grupo.

Na última etapa, os resultados da aplicação da função em cada um dos grupos são combinados em um objeto de resultado (*combine*).


As chaves que agrupamos podem ser especificadas de várias maneiras diferentes:

* Uma lista ou numpy array do mesmo tamanho que o eixo selecionado

* Para objetos DataFrame, uma string que indica o nome da coluna pela qual vamos agrupar.

* Para objetos DataFrame, uma string que indica o nome do índice pelo qual vamos agrupar.

* Um dicionário ou Series que estabelece um mapeamento entre um valor e o nome do grupo.

* Uma função python que será avaliada em cada um dos rótulos dos eixos.

* Uma lista com qualquer uma das opções acima.


Observe que o resultado de cada uma dessas opções é **produzir uma matriz de valores que usaremos para dividir** o objeto Series ou DataFrame.


Vamos ler os dados da pesquisa de turismo GCBA, ver o quão grande é o DataFrame, quais colunas ele tem, que tipo de dados ele é e os primeiros registros lidos.

In [55]:
import pandas as pd
import numpy as np

In [56]:
# low_memorybool, default True
# Internally process the file in chunks, resulting in lower memory use while parsing, 
# but possibly mixed type inference. To ensure no mixed types either set False, 
# or specify the type with the dtype parameter. 
# Note that the entire file is read into a single DataFrame regardless, 
# use the chunksize or iterator parameter to return the data in chunks. (Only valid with C parser).

data = pd.read_csv("./dados/resultado-de-encuestas-2017-2018.csv", sep = ",", low_memory = False) 

print(data.shape)
print(data.columns)
print(data.dtypes)

data.head(3)

(1105, 20)
Index(['id', 'fecha', 'centro_atencion_turistica', 'barrio', 'comuna',
       'pasajeros', 'pais_residencia_si_extranjero',
       'otro_pais_residencia_si_extranjero',
       'provincia_residencia_si_argentino', 'pernoctaciones',
       'medio_transporte_llegada', 'alojamiento', 'otro_alojamiento',
       'barrio.1', 'otro_barrio', 'primera_vez', 'motivo_viaje',
       'otro_motivo_viaje', 'motivo_consulta', 'otro_motivo_consulta'],
      dtype='object')
id                                      int64
fecha                                  object
centro_atencion_turistica              object
barrio                                 object
comuna                                 object
pasajeros                               int64
pais_residencia_si_extranjero          object
otro_pais_residencia_si_extranjero     object
provincia_residencia_si_argentino      object
pernoctaciones                        float64
medio_transporte_llegada               object
alojamiento            

,id,fecha,centro_atencion_turistica,barrio,comuna,pasajeros,pais_residencia_si_extranjero,otro_pais_residencia_si_extranjero,provincia_residencia_si_argentino,pernoctaciones,medio_transporte_llegada,alojamiento,otro_alojamiento,barrio.1,otro_barrio,primera_vez,motivo_viaje,otro_motivo_viaje,motivo_consulta,otro_motivo_consulta
0,1,2017-01-02,retiro,RETIRO,COMUNA 1,2,Chile,NaN,NaN,7.0,No especifica,NaN,NaN,NaN,NaN,No especifica,No especifica,NaN,Mapas Orientacion alojamiento,NaN
1,2,2017-01-02,retiro,RETIRO,COMUNA 1,1,NaN,NaN,Provincia de Buenos Aires,2.0,No especifica,NaN,NaN,NaN,NaN,No especifica,No especifica,NaN,Mapas Orientacion,NaN
2,3,2017-01-02,retiro,RETIRO,COMUNA 1,2,NaN,NaN,Córdoba,0.0,No especifica,No pernocta en Buenos Aires,NaN,NaN,NaN,No especifica,No especifica,NaN,Mapas Orientacion alrededores,NaN


Vamos ver qual porcentagem de registros nulos em cada coluna:

In [57]:
data.isnull().sum() / data.shape[0]

id                                    0.000000
fecha                                 0.000000
centro_atencion_turistica             0.000000
barrio                                0.000000
comuna                                0.000000
pasajeros                             0.000000
pais_residencia_si_extranjero         0.386425
otro_pais_residencia_si_extranjero    0.993665
provincia_residencia_si_argentino     0.629864
pernoctaciones                        0.687783
medio_transporte_llegada              0.000000
alojamiento                           0.259729
otro_alojamiento                      1.000000
barrio.1                              0.959276
otro_barrio                           1.000000
primera_vez                           0.000000
motivo_viaje                          0.051584
otro_motivo_viaje                     1.000000
motivo_consulta                       0.000000
otro_motivo_consulta                  0.994570
dtype: float64

Vemos que existem várias colunas com uma porcentagem muito alta de valores nulos.

## Agregações simples

### Pergunta 1:

A coluna `['pernoctaciones']` é do tipo numérica (float64), vamos calcular quantos dias no total e em média dura a viagem das pessoas que vieram a estes centros de atendimento.

In [58]:
pernoites_serie = data.pernoctaciones
print("Duração média da viagem:", pernoites_serie.mean())
print("Duração total da viagem:", pernoites_serie.sum())

Duração média da viagem: 5.3826086956521735
Duração total da viagem: 1857.0


### Pergunta 2:

**2.a** Quantos e quais são os motivos da consulta?

Vamos analisar o campo `motivo_consulta` que não possui valores nulos.

Ajuda: [`pandas.Series.value_counts()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html)

**2.b** Quais são os 5 motivos mais consultados?

Ajuda: Vamos indexar a Series ordenada que é o resultado de value_counts.

(value_counts retorna a Series ordenada).

In [59]:
motivos = data.motivo_consulta.value_counts()
motivos.head(3)

motivo_consulta
Bus                  169
Orientacion          154
Mapas Orientacion    114
Name: count, dtype: int64

In [60]:
motivos_top5 = motivos[0 : 5]
motivos_top5

motivo_consulta
Bus                  169
Orientacion          154
Mapas Orientacion    114
Mapas                 97
Bus Mapas             35
Name: count, dtype: int64

### Pergunta 3:

Usando o método [`pandas.DataFrame.describe()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html), vamos avaliar as colunas pasajeros (passageiros) e pernoctaciones (pernoites).

Este método retorna um DataFrame, usando esse resultado vamos responder qual é o número médio de passageiros.

In [61]:
medidas = data[["pasajeros", "pernoctaciones"]].describe()
print(type(medidas))
print("Número médio de passageiros: ", medidas.loc["mean", "pasajeros"].round(2))
medidas

<class 'pandas.core.frame.DataFrame'>
Número médio de passageiros:  2.01


,pasajeros,pernoctaciones
count,1105.000000,345.000000
mean,2.011765,5.382609
std,1.172931,10.865656
min,1.000000,0.000000
25%,1.000000,3.000000
50%,2.000000,4.000000
75%,2.000000,6.000000
max,12.000000,180.000000


Vemos que o número máximo de dormidas é de 690 e o número máximo de passageiros é de 150.

Vamos ver quais registros têm esses valores. E vamos tentar entender se correspondem ou não a um erro.

In [62]:
data_passageiros_150_mask = data.pasajeros == 150
data_passageiros_150 = data.loc[data_passageiros_150_mask, ]
data_passageiros_150

,id,fecha,centro_atencion_turistica,barrio,comuna,pasajeros,pais_residencia_si_extranjero,otro_pais_residencia_si_extranjero,provincia_residencia_si_argentino,pernoctaciones,medio_transporte_llegada,alojamiento,otro_alojamiento,barrio.1,otro_barrio,primera_vez,motivo_viaje,otro_motivo_viaje,motivo_consulta,otro_motivo_consulta


Parece ser um contingente de estudantes franceses em trânsito ("Não pernoite em Buenos Aires").

In [63]:
data_pernoites_690_mask = data.pernoctaciones == 690
data_pernoites_690 = data.loc[data_pernoites_690_mask, ]
data_pernoites_690

,id,fecha,centro_atencion_turistica,barrio,comuna,pasajeros,pais_residencia_si_extranjero,otro_pais_residencia_si_extranjero,provincia_residencia_si_argentino,pernoctaciones,medio_transporte_llegada,alojamiento,otro_alojamiento,barrio.1,otro_barrio,primera_vez,motivo_viaje,otro_motivo_viaje,motivo_consulta,otro_motivo_consulta


Desse cadastro não podemos extrair informações que justifiquem 690 dias de permanência.

##  Como construímos grupos?


### DataFrameGroupBy


A definição abstrata de agrupamento é fornecer um mapeamento entre valores e (rótulos ou) nomes de grupos.

Um objeto [`pandas.DataFrame.groupby`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) não calcula nada, mas cria uma estrutura intermediária com todas as informações necessárias para posteriormente aplicar alguma operação a cada grupo.

O **resultado dessa operação** é retornado em um Series ou DataFrame **indexado pelos valores exclusivos da key groupby**.

As operações que podemos aplicar em um objeto `groupby` estão listadas [aqui](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html) Usando o resultado de uma operação de agrupamento por nome de coluna como exemplo, vamos apresentar algumas propriedades e métodos deste objeto.

Vamos agrupar os dados por coluna em `pais_residencia_si_extranjero`.

#### Tipo 

Vamos ver que tipo é o objeto retornado:


In [64]:
data_grouped = data.groupby('pais_residencia_si_extranjero')
type(data_grouped)

pandas.core.groupby.generic.DataFrameGroupBy

#### size

Vamos ver o número de registros em cada grupo e quantos registros do `DataFrame` original são atribuídos a algum grupo com o atributo [`.size()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.size.html).

In [65]:
data_grouped.size()

pais_residencia_si_extranjero
Alemania                                     25
Australia                                    12
Bolivia                                      11
Brasil                                      213
Canadá                                        7
Chile                                        31
China                                        14
Colombia                                     71
Corea del Sur                                 1
Costa Rica                                    5
Ecuador                                       8
España                                       54
Estados Unidos                               50
Francia                                      37
India                                         2
Israel                                        5
Italia                                       29
Japón                                         1
México                                       10
Noruega                                       4
Otro país 

In [66]:
data_grouped.size().sum()

678

In [67]:
data_grouped.size().sum() / data.shape[0]

0.6135746606334842

Apenas $52\%$ dos registros foram atribuídos a um grupo.

Vamos ver se esse número de registros atribuídos a um grupo corresponde ao número de registros não nulos nesse campo.

In [68]:
data.loc[data.pais_residencia_si_extranjero.notnull(), ].shape[0]

678

#### Índices

É um dicionário cujas chaves são os valores únicos das chaves groupby, neste caso os valores da coluna `country_residence_if_extranjero`, e cujos valores são uma matriz com os [índices](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.indices.html#pandas.core.groupby.GroupBy.indices) do DataFrame cujo valor nessa coluna é igual ao da chave.

In [69]:
print(type(data_grouped.indices))
#data_grouped.indices

<class 'dict'>


Neste exemplo, os índices 48, 108, 147, ... correspondem à Alemanha; os índices 33, 242, 267, ... correspondem à Austrália, os índices 6, 8, 28, ... ao Uruguai. Vamos verificar isso:

In [70]:
data.pais_residencia_si_extranjero.loc[[48, 108, 147]]

48     Alemania
108    Alemania
147    Alemania
Name: pais_residencia_si_extranjero, dtype: object

In [71]:
data.pais_residencia_si_extranjero.loc[[33, 242, 267]]

33     Australia
242    Australia
267    Australia
Name: pais_residencia_si_extranjero, dtype: object

In [72]:
data.pais_residencia_si_extranjero.loc[[ 6, 8, 28]]

6     Uruguay
8     Uruguay
28    Uruguay
Name: pais_residencia_si_extranjero, dtype: object

#### groups

É semelhante a `index`, mas associando as chaves do groupby a objetos do tipo Index.

In [73]:
#data_grouped.groups

### Groupby com dicionários e Series como chave

Até agora vimos como definir uma ou mais colunas de um DataFrame como a key do groupby. Quando fazemos isso, estamos definindo para cada registro um rótulo (o valor do campo ou campos-chave nesse registro) que usaremos para determinar a qual grupo ele pertence.

A seguir, veremos exemplos de Series e dicionários como chaves do groupby.

Para agrupar usando uma instância de Series ou dicionário como chave, precisamos que os valores do índice da Series ou das chaves do dicionário sejam os mesmos do índice do DataFrame no qual queremos grupo.

Vamos definir um dicionário que associa os países ao seu contêiner:

In [74]:
pais_em_continente = {
    'Chile': 'America', 'Francia': 'Europa', 'México': 'America', 'Colombia': 'America', 
    'Uruguay': 'America', 'Estados Unidos': 'America', 'España': 'Europa', 'Italia': 'Europa', 
    'India': 'Asia', 'Ecuador': 'America', 'Brasil': 'America',
    'Australia': 'Oceania', 'Bolivia': 'America', 'Reino Unido': 'Europa', 
    'Alemania': 'Europa', 'Israel': 'Asia', 'China': 'Asia', 
    'Venezuela': 'America', 'Países Bajos': 'Europa', 'Canadá': 'America', 'Suiza': 'Europa', 'Turquía': 'Europa',
    'Noruega': 'Europa', 'Corea del Sur': 'Asia', 'Polonia': 'Europa', 'Perú': 'America', 'Paraguay': 'America',
    'Costa Rica': 'America', 'Japón': 'Asia', 'Marruecos': 'Africa', 'Bélgica': 'Europa', 'Malasia': 'Asia', 
    'Rusia': 'Europa', 'Sudáfrica': 'Africa', 'Nueva Zelanda': 'Oceania'        
}

Vamos atribuir como índice do DataFrame os valores do campo pais_residencia_si_extranjero, que coincidem com as chaves do dicionário que definimos acima.

In [75]:
data.index = data.pais_residencia_si_extranjero

Vamos contar quantos turistas de cada continente solicitaram informações no posto de turismo.

In [76]:
data_grouped_continente = data.groupby(pais_em_continente)
data_grouped_continente["pasajeros"].sum()

pais_residencia_si_extranjero
America    1028
Asia         38
Europa      387
Oceania      23
Name: pasajeros, dtype: int64

Da mesma forma, podemos indexar Series com dicionários:

In [77]:
# criamos a série
serie_passageiros = data.pasajeros

# atribuímos como índice o valor do campo pais_residencia_si_extranjero para esse registro
serie_passageiros.index = data.pais_residencia_si_extranjero

# agrupamos e somamos
serie_passageiros.groupby(pais_em_continente).sum()

pais_residencia_si_extranjero
America    1028
Asia         38
Europa      387
Oceania      23
Name: pasajeros, dtype: int64

Agora queremos indexar um DataFrame com um objeto Series.

Vamos transformar o dicionário pais_en_continente em uma instância Series e usá-lo para indexar.

In [78]:
pais_em_continente_serie = pd.Series(pais_em_continente)
pais_em_continente_serie

Chile             America
Francia            Europa
México            America
Colombia          America
Uruguay           America
Estados Unidos    America
España             Europa
Italia             Europa
India                Asia
Ecuador           America
Brasil            America
Australia         Oceania
Bolivia           America
Reino Unido        Europa
Alemania           Europa
Israel               Asia
China                Asia
Venezuela         America
Países Bajos       Europa
Canadá            America
Suiza              Europa
Turquía            Europa
Noruega            Europa
Corea del Sur        Asia
Polonia            Europa
Perú              America
Paraguay          America
Costa Rica        America
Japón                Asia
Marruecos          Africa
Bélgica            Europa
Malasia              Asia
Rusia              Europa
Sudáfrica          Africa
Nueva Zelanda     Oceania
dtype: object

In [79]:
data.index = data.pais_residencia_si_extranjero
data_grouped_continente_2 = data.groupby(pais_em_continente_serie)
data_grouped_continente_2["pasajeros"].sum()

America    1028
Asia         38
Europa      387
Oceania      23
Name: pasajeros, dtype: int64

## Groupby com funções

Qualquer função que passarmos como uma chave de grupo será chamada uma vez para cada valor do índice e o resultado será o nome do grupo.

Vejamos um exemplo:

Definimos uma função que, dada uma string que representa um país, retorna o nome do contêiner daquele país:

In [80]:
def get_continente(pais):
    pais_em_continente = {
    'Chile': 'America', 'Francia': 'Europa', 'México': 'America', 'Colombia': 'America', 
    'Uruguay': 'America', 'Estados Unidos': 'America', 'España': 'Europa', 'Italia': 'Europa', 
    'India': 'Asia', 'Ecuador': 'America', 'Brasil': 'America',
    'Australia': 'Oceania', 'Bolivia': 'America', 'Reino Unido': 'Europa', 
    'Alemania': 'Europa', 'Israel': 'Asia', 'China': 'Asia', 
    'Venezuela': 'America', 'Países Bajos': 'Europa', 'Canadá': 'America', 'Suiza': 'Europa', 'Turquía': 'Europa',
    'Noruega': 'Europa', 'Corea del Sur': 'Asia', 'Polonia': 'Europa', 'Perú': 'America', 'Paraguay': 'America',
    'Costa Rica': 'America', 'Japón': 'Asia', 'Marruecos': 'Africa', 'Bélgica': 'Europa', 'Malasia': 'Asia', 
    'Rusia': 'Europa', 'Sudáfrica': 'Africa', 'Nueva Zelanda': 'Oceania'}
    if pais in pais_em_continente:
        result = pais_em_continente[pais]
    else:
        result = "desconocido"
    return result
    

Repetimos o exercício anterior agrupando com esta função.

Lembre-se que **a função que é a chave do groupby recebe como argumento o valor do índice de cada registro** quando axis = 0 (que é o valor padrão do eixo) e recebe o valor da coluna quando axis = 1.

Em todos os exercícios desta prática, agrupamos por linhas (axis = 0), mas a mesma lógica se aplica se quisermos agrupar por colunas.

In [81]:
data.index = data.pais_residencia_si_extranjero
data_grouped_func = data.groupby(get_continente, axis = 0)
data_grouped_func["pasajeros"].sum()

C:\Users\costa\AppData\Local\Temp\ipykernel_26296\3354891323.py:2: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  data_grouped_func = data.groupby(get_continente, axis = 0)


pais_residencia_si_extranjero
America        1028
Asia             38
Europa          387
Oceania          23
desconocido     747
Name: pasajeros, dtype: int64

## Que operações podemos fazer em grupos?

In [82]:
# redefinimos o índice de dados, que modificamos nos exercícios anteriores

data = data.reset_index(drop = True)
data_grouped = data.groupby('pais_residencia_si_extranjero')

### Estatísticas descritivas sobre grupos

Vamos calcular o número de turistas de cada país que solicitaram informações.

Para tanto vamos selecionar a coluna `passageiros` do objeto [`DataFrameGroupBy`](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html#computations-descriptive-stats) e adicionar esse campo nos registros que compõem cada grupo.

In [83]:
sum_por_pais = data_grouped["pasajeros"].sum()
print(type(sum_por_pais))
sum_por_pais

<class 'pandas.core.series.Series'>


pais_residencia_si_extranjero
Alemania                                     52
Australia                                    23
Bolivia                                      28
Brasil                                      501
Canadá                                       12
Chile                                        65
China                                        20
Colombia                                    161
Corea del Sur                                 1
Costa Rica                                   10
Ecuador                                      19
España                                      118
Estados Unidos                              101
Francia                                      82
India                                         4
Israel                                       11
Italia                                       55
Japón                                         2
México                                       25
Noruega                                      10
Otro país 

Vemos que o resultado é um objeto do tipo Series e seu índice são os valores únicos do campo que usamos como a chave do groupby.

Agora queremos ver um ranking dos países com base no número de turistas que visitam a cidade de Buenos Aires. Para isso ordenamos a série resultante do ponto anterior do maior para o menor, usando o método [`pandas.Series.sort_values()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.sort_values.html).

In [84]:
sum_por_pais_sorted = sum_por_pais.sort_values(ascending = False)
sum_por_pais_sorted

pais_residencia_si_extranjero
Brasil                                      501
Colombia                                    161
España                                      118
Estados Unidos                              101
Francia                                      82
Chile                                        65
Italia                                       55
Alemania                                     52
Reino Unido                                  50
Uruguay                                      39
Bolivia                                      28
Perú                                         26
Venezuela                                    26
México                                       25
Australia                                    23
China                                        20
Ecuador                                      19
Paraguay                                     15
Canadá                                       12
Israel                                       11
Noruega   

Calculemos agora a média e o desvio padrão das dormidas por país.

Já calculamos groupby por país e atribuímos à variável [`data_grouped`](https://realpython.com/pandas-groupby/), selecionamos o campo pernoites e calculamos essas medidas.

- [`pandas.core.groupby.GroupBy.mean()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.mean.html#pandas.core.groupby.GroupBy.mean)

- [`pandas.core.groupby.GroupBy.std()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.std.html#pandas.core.groupby.GroupBy.std)

Também podemos usar [`pandas.core.groupby.DataFrameGroupBy.describe()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.DataFrameGroupBy.describe.html) sobre os grupos.

In [85]:
data_grouped["pernoctaciones"].mean()

pais_residencia_si_extranjero
Alemania                                     4.600000
Australia                                    3.666667
Bolivia                                     40.400000
Brasil                                       4.696970
Canadá                                      25.500000
Chile                                        4.083333
China                                        5.000000
Colombia                                     6.000000
Corea del Sur                                     NaN
Costa Rica                                   5.500000
Ecuador                                      5.500000
España                                       4.428571
Estados Unidos                               4.800000
Francia                                      4.466667
India                                        3.000000
Israel                                       2.500000
Italia                                       4.400000
Japón                                             Na

In [86]:
data_grouped["pernoctaciones"].std()

pais_residencia_si_extranjero
Alemania                                     2.319004
Australia                                    1.154701
Bolivia                                     78.082008
Brasil                                       2.287743
Canadá                                      43.023250
Chile                                        1.880925
China                                             NaN
Colombia                                     2.904270
Corea del Sur                                     NaN
Costa Rica                                   0.707107
Ecuador                                      0.707107
España                                       1.776835
Estados Unidos                               4.538722
Francia                                      1.684665
India                                             NaN
Israel                                       2.886751
Italia                                       1.298351
Japón                                             Na

In [87]:
data_grouped["pernoctaciones"].describe()

,count,mean,std,min,25%,50%,75%,max
pais_residencia_si_extranjero,,,,,,,,
Alemania,10.0,4.600000,2.319004,2.0,3.25,4.5,5.00,10.0
Australia,3.0,3.666667,1.154701,3.0,3.00,3.0,4.00,5.0
Bolivia,5.0,40.400000,78.082008,2.0,4.00,8.0,8.00,180.0
Brasil,99.0,4.696970,2.287743,1.0,3.00,5.0,6.00,15.0
Canadá,4.0,25.500000,43.023250,3.0,3.00,4.5,27.00,90.0
Chile,12.0,4.083333,1.880925,1.0,3.00,4.0,5.25,7.0
China,1.0,5.000000,NaN,5.0,5.00,5.0,5.00,5.0
Colombia,24.0,6.000000,2.904270,2.0,4.00,5.5,7.00,15.0
Corea del Sur,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Índices hierárquicos

Podemos agrupar por mais de um campo, e o resultado será uma Series ou DataFrame com um índice hierárquico definido pelos campos-chave do agrupamento.

Vamos ver como o número de turistas é distribuído por país de residência por bairro.

In [88]:
data_grouped_pais_bairro = data.groupby(["pais_residencia_si_extranjero", "barrio"])
quantidade_pasajeros_pais_bairro = data_grouped_pais_bairro["pasajeros"].sum()
quantidade_pasajeros_pais_bairro

pais_residencia_si_extranjero  barrio       
Alemania                       PUERTO MADERO    11
                               RECOLETA          4
                               RETIRO            2
                               SAN NICOLAS      35
Australia                      PUERTO MADERO     3
                                                ..
Uruguay                        RETIRO            1
                               SAN NICOLAS      32
Venezuela                      PALERMO           2
                               PUERTO MADERO     8
                               SAN NICOLAS      16
Name: pasajeros, Length: 99, dtype: int64

Vemos que o índice do objeto Result Series possui dois níveis. Se quisermos ver como foi definido:

In [89]:
quantidade_pasajeros_pais_bairro.index

MultiIndex([(                                'Alemania', 'PUERTO MADERO'),
            (                                'Alemania',      'RECOLETA'),
            (                                'Alemania',        'RETIRO'),
            (                                'Alemania',   'SAN NICOLAS'),
            (                               'Australia', 'PUERTO MADERO'),
            (                               'Australia',      'RECOLETA'),
            (                               'Australia',   'SAN NICOLAS'),
            (                                 'Bolivia', 'PUERTO MADERO'),
            (                                 'Bolivia',        'RETIRO'),
            (                                 'Bolivia',   'SAN NICOLAS'),
            (                                  'Brasil',          'BOCA'),
            (                                  'Brasil',       'PALERMO'),
            (                                  'Brasil', 'PUERTO MADERO'),
            (            

Podemos usar o método [`pandas.Series.unstack()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unstack.html) para criar um DataFrame a partir deste objeto Series.

In [90]:
quantidade_pasajeros_pais_bairro_df = quantidade_pasajeros_pais_bairro.unstack()
print(type(quantidade_pasajeros_pais_bairro_df))
quantidade_pasajeros_pais_bairro_df.head(3)

<class 'pandas.core.frame.DataFrame'>


barrio,BOCA,PALERMO,PUERTO MADERO,RECOLETA,RETIRO,SAN NICOLAS
pais_residencia_si_extranjero,,,,,,
Alemania,NaN,NaN,11.0,4.0,2.0,35.0
Australia,NaN,NaN,3.0,6.0,NaN,14.0
Bolivia,NaN,NaN,5.0,NaN,6.0,17.0



<a id="section_aggregate_transform_filter"></a> 
### Aggregate, transform, filter


Uma vez construídos os grupos (como resultado da etapa `Split`), na etapa `Apply` podemos realizar as operações sobre eles, quais sejam:

* **agregação**: cálculo das estatísticas de resumo para cada grupo. Por exemplo, soma ou média

* **transformação**: cálculos específicos do grupo retornando novos objetos indexados da mesma maneira. Por exemplo, preencha os NAs dentro de um grupo com um valor calculado nesse grupo, como média, mediana, máximo, etc.

* **filtro**: descarte alguns grupos de acordo com algum cálculo no grupo que retorna Verdadeiro ou Falso. Por exemplo, descarte grupos com poucos membros.

Exemplos de operações de agregação são todos os que vimos até agora. Depois de construir um grupo com qualquer uma das alternativas que apresentamos, calculamos uma medida em cada um desses grupos.

#### Transformação

Vimos que no domínio das dormidas existe uma percentagem muito elevada de nulos.

Vamos completar os valores deste campo, atribuindo a média das dormidas agrupadas por `pais_residencia_si_extranjero` e `bairro`.

Sabemos que `groupby` não configura grupos definidos por nulos, portanto aqueles registros que possuem `null` nos campos que são chave para `groupby` não serão atribuídos a nenhum grupo.

Antes de começar, vamos remover esses registros.

In [91]:
data_key_not_null_mask = np.logical_and(data.pais_residencia_si_extranjero.notnull(), data.barrio.notnull())
data_key_not_null = data.loc[data_key_not_null_mask, :]
data_key_not_null.shape

(678, 20)

Re-definimos os índices com a função [`reset_index()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html)

In [92]:
# se necessário, eliminamos o índice que atribuímos aos exercícios agrupados por série ou dicionário:
data_key_not_null = data_key_not_null.reset_index(drop = True)

#data_key_not_null.head(3)

In [93]:
data_key_not_null_grouped_pais_bairro = data_key_not_null.groupby(["pais_residencia_si_extranjero", "barrio"])

Qual porcentagem de valores nulos estão na coluna pernoctaciones em data_key_not_null?

In [94]:
data_key_not_null["pernoctaciones"].isnull().sum() / data_key_not_null.shape[0]

0.6017699115044248

Usamos [`transform`](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#transformation) para preencher os valores nulos com a média por grupo e contamos quantos valores nulos restam.

In [95]:
data_filled = data_key_not_null_grouped_pais_bairro["pernoctaciones"].transform(lambda grp: grp.fillna(grp.mean()))
data_filled

0      7.000000
1      4.000000
2      0.000000
3      6.375000
4           NaN
         ...   
673    4.000000
674    5.000000
675    6.000000
676    5.500000
677    5.533333
Name: pernoctaciones, Length: 678, dtype: float64

In [96]:
data_filled.isnull().sum()

82

Vemos que existem 10 registros restantes que foram atribuídos a um grupo, mas eles ainda são nulos. **O que aconteceu?**

Vejamos o que são esses registros e quais valores eles têm nos campos "pais_residencia_si_extranjero" e "barrio".

In [97]:
data_not_filled = data_filled.loc[data_filled.isnull()]

In [98]:
data_key_not_null.loc[data_not_filled.index, [ "pais_residencia_si_extranjero", "barrio"]]

,pais_residencia_si_extranjero,barrio
4,Uruguay,SAN NICOLAS
6,Uruguay,SAN NICOLAS
17,Uruguay,SAN NICOLAS
40,India,SAN NICOLAS
42,Israel,SAN NICOLAS
...,...,...
648,Uruguay,SAN NICOLAS
653,Uruguay,SAN NICOLAS
658,México,RECOLETA
660,Uruguay,SAN NICOLAS


Miremos los valores en el campo "pernoctaciones" de los registros de los grupos 
Vejamos os valores no campo "pernoctaciones" dos registros de grupo
* Malasia	RECOLETA	
* Marruecos	PALERMO	
* Estados Unidos	NÃO IDENTIFICADO
* Brasil	NÃO IDENTIFICADO

In [99]:
malasia_recoleta_mask = np.logical_and(data_key_not_null.pais_residencia_si_extranjero  == 'Malasia', 
                                        data_key_not_null.barrio == "RECOLETA")

data_key_not_null.loc[malasia_recoleta_mask, "pernoctaciones"]

Series([], Name: pernoctaciones, dtype: float64)

In [100]:
marruecos_palermo_mask = np.logical_and(data_key_not_null.pais_residencia_si_extranjero  == 'Marruecos', 
                                        data_key_not_null.barrio == "PALERMO")

data_key_not_null.loc[marruecos_palermo_mask, "pernoctaciones"]

Series([], Name: pernoctaciones, dtype: float64)

In [101]:
usa_sinid_mask = np.logical_and(data_key_not_null.pais_residencia_si_extranjero  == 'Estados Unidos', 
                                        data_key_not_null.barrio == "SIN IDENTIFICAR")

data_key_not_null.loc[usa_sinid_mask, "pernoctaciones"]

Series([], Name: pernoctaciones, dtype: float64)

In [102]:
brasil_sinid_mask = np.logical_and(data_key_not_null.pais_residencia_si_extranjero  == 'Brasil', 
                                        data_key_not_null.barrio == "SIN IDENTIFICAR")

data_key_not_null.loc[brasil_sinid_mask, "pernoctaciones"]

Series([], Name: pernoctaciones, dtype: float64)

Vemos que todos os registros desses grupos têm valor nulo no campo de pernoites, portanto a média por grupo também é nula e novamente temos nulo como preenchimento por grupo.

#### Filtro

O método [`filtration`](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#filtration) retorna um subconjunto do objeto original.

Suponha que desejamos retornar os registros que correspondem a países com mais de 1000 visitas.

In [103]:
data_group_pais = data.groupby(data.pais_residencia_si_extranjero)
data_group_pais.size()

pais_residencia_si_extranjero
Alemania                                     25
Australia                                    12
Bolivia                                      11
Brasil                                      213
Canadá                                        7
Chile                                        31
China                                        14
Colombia                                     71
Corea del Sur                                 1
Costa Rica                                    5
Ecuador                                       8
España                                       54
Estados Unidos                               50
Francia                                      37
India                                         2
Israel                                        5
Italia                                       29
Japón                                         1
México                                       10
Noruega                                       4
Otro país 

In [104]:
data_paises_frequentes = data_group_pais.filter(lambda grp: grp["pasajeros"].sum() > 1000)

Tamanho antes do filtro (também estamos contando registros que têm nulos em pais_residencia_si_extranjero):

In [105]:
data.shape

(1105, 20)

Tamanho após o filtro (apenas registros não nulos no campo pais_residencia_si_extranjero):

In [106]:
data_paises_frequentes.shape

(0, 20)

Outra maneira de calcular os tamanhos antes do filtro:

(apenas registros que não são nulos no campo pais_residencia_si_extranjero)

In [107]:
data_group_pais.size().sum()

678

#### Apply

Podemos avaliar/aplicar funções em grupos usando [`apply`](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#flexible-apply).

Vejamos um exemplo em que aplicamos o método [`pandas.core.groupby.DataFrameGroupBy.describe()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.DataFrameGroupBy.describe.html) em cada um dos grupos por país.

In [108]:
data_group_pais = data.groupby(data.pais_residencia_si_extranjero)
data_group_pais.apply(lambda grp: grp.describe())

C:\Users\costa\AppData\Local\Temp\ipykernel_26296\19801275.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_group_pais.apply(lambda grp: grp.describe())


id  pasajeros  pernoctaciones  \
pais_residencia_si_extranjero                                                 
Alemania                      count    25.000000  25.000000       10.000000   
                              mean    607.160000   2.080000        4.600000   
                              std     295.109963   1.077033        2.319004   
                              min      49.000000   1.000000        2.000000   
                              25%     335.000000   2.000000        3.250000   
...                                          ...        ...             ...   
Venezuela                     min      78.000000   1.000000        3.000000   
                              25%     266.000000   2.000000        4.000000   
                              50%     543.000000   2.000000        5.000000   
                              75%     781.000000   2.000000        6.000000   
                              max    1055.000000   4.000000        7.000000   

                                     otro_alojamiento  otro_barrio  \
pais_residencia_si_extranjero                                        
Alemania                      count               0.0          0.0   
                              mean                NaN          NaN   
                              std                 NaN          NaN   
                              min                 NaN          NaN   
                              25%                 NaN          NaN   
...                                               ...          ...   
Venezuela                     min                 NaN          NaN   
                              25%                 NaN          NaN   
                              50%                 NaN          NaN   
                              75%                 NaN          NaN   
                              max                 NaN          NaN   

                                     otro_motivo_viaje  
pais_residencia_si_extranjero                           
Alemania                      count                0.0  
                              mean                 NaN  
                              std                  NaN  
                              min                  NaN  
                              25%                  NaN  
...                                                ...  
Venezuela                     min                  NaN  
                              25%                  NaN  
                              50%                  NaN  
                              75%                  NaN  
                              max                  NaN  

[240 rows x 6 columns]

#### Referências

Python for Data Analysis. Wes McKinney. Cap 10

- [Group by: split-apply-combine](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)

- [Grouping](https://pandas.pydata.org/pandas-docs/stable/user_guide/cookbook.html#cookbook-grouping)